1. 

Készítsünk sorszámozott listát a szálláshelyek adatairól! 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
SELECT *,
       ROW_NUMBER() OVER(PARTITION BY tipus 
           ORDER BY tipus, hely, szallas_nev)
FROM szallashely

3\. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

In [ ]:
SELECT u.[LOGIN],
       u.NEV,
       f.METTOL,
       f.MEDDIG,
       f.FELNOTT_SZAM + f.GYERMEK_SZAM,
       -- új oszlop: az előző 3 foglalás összesen
       SUM(f.FELNOTT_SZAM + f.GYERMEK_SZAM) 
          OVER(PARTITION BY u.LOGIN ORDER BY u.login, f.METTOL 
               ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING)

FROM Foglalas f JOIN Ugyfel u ON f.UGYFEL_FK = u.[LOGIN]

6. 

<span style="background-color: rgba(220, 220, 220, 0.4); white-space-collapse: preserve;"><font color="#000000">SELECT rt.TERMEKKOD, </font></span> <span style="color: var(--vscode-editor-foreground); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px; background-color: rgba(220, 220, 220, 0.4); white-space-collapse: preserve;">AVG(rt.EGYSEGAR) AS '�tlagos �r' </span> <span style="color: var(--vscode-editor-foreground); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px; background-color: rgba(220, 220, 220, 0.4); white-space-collapse: preserve;">FROM Rendeles_tetel rt JOIN Rendeles r </span> <span style="color: var(--vscode-editor-foreground); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px; background-color: rgba(220, 220, 220, 0.4); white-space-collapse: preserve;">ON rt.SORSZAM = r.SORSZAM </span> <span style="color: var(--vscode-editor-foreground); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px; background-color: rgba(220, 220, 220, 0.4); white-space-collapse: preserve;">GROUP BY rt.TERMEKKOD</span>

In [ ]:
SELECT rt.TERMEKKOD, 
       AVG(rt.EGYSEGAR) AS '�tlagos �r'
FROM Rendeles_tetel rt 
CROSS APPLY
(    SELECT *
     FROM Rendeles r
     WHERE  rt.SORSZAM = r.SORSZAM
)r
GROUP BY rt.TERMEKKOD

7. <span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Alakítsa át a melléklet 2. pontjában lévő lekérdezést úgy, hogy mindkét (INNER) JOIN részbe sorra beilleszti a tanult JOIN HINT-eket (először mindkét helyre MERGE, majd HASH, utána LOOP, végül REMOTE).</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Melyik hint használata adja a legjobb megoldást?</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. Mi lesz ezt a hint-et tartalmazó lekérdezés kódja?</span>

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);"><br></span>

In [ ]:
SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendel�si t�telek �ssz�rt�ke'
FROM Rendeles_tetel rt INNER HASH JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 INNER HASH JOIN Ugyfel u ON r.LOGIN = u.LOGIN

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">8. Készítsen non-clustered indexet a Rendeles_tetel táblához!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Az index neve NCI_sorszam_termekkod legyen</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. A sorszam legyen az index kulcs oszlop, a termékkód pedig az indexbe belefoglalt oszlop</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">c. Válaszként adja meg az indexet generáló script-et!</span>

In [ ]:
--9 feladat: TABLE HINT index-hez

SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendel�si t�telek �ssz�rt�ke'
FROM Rendeles_tetel rt JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 JOIN Ugyfel u ON r.LOGIN = u.LOGIN
OPTION(TABLE HINT(rt, INDEX(NCI_sorszam_termekkod)))

SORSZAM,TERMEKKOD,EGYSEGAR,MENNYISEG,nev,Eddigi rendel�si t�telek �ssz�rt�ke
121,01080093T,146,20,Kiss Ádám,2920
121,02020444T,421,60,Kiss Ádám,28180
121,02060034T,270,40,Kiss Ádám,38980
121,04030287T,20,90,Kiss Ádám,40780
121,07040134T,40,90,Kiss Ádám,44380
121,07080421T,207,10,Kiss Ádám,46450
121,08070483T,294,50,Kiss Ádám,61150
207,F021-IN215,5900,4,Kiss Ádám,84750
207,G019-NA207,1172,9,Kiss Ádám,95298
207,G082-CI419,7272,1,Kiss Ádám,102570
